HRC Case with instantaneus action.

In [1]:
!pip install --pre unified-planning[fmap]
!apt install libgraphviz-dev
!pip install pygraphviz

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.23ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: gpg 1.13.1-unknown has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gpg or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.36ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that t

In [4]:
from unified_planning.shortcuts import *
from unified_planning.model.multi_agent import *
from collections import namedtuple
from unified_planning.io.ma_pddl_writer import MAPDDLWriter

# Parameters
locations_name = ["pick_area", "release_area"]

obj_types = ["w","b","o"]
n_4_type = 4

objects_name = [f"{obj_type}{k}" for obj_type in obj_types for k in range(1,n_4_type + 1)]
obj_goals = ["b1","b2","b3","b4","o1","o2","o3","o4","w1","w3","w2","w4"]
obj_agents = {"w":"robot",
             "o":"human",
             "b1":"robot",
             "b2":"robot",
             "b3":"human",
             "b4":"human"}

# Problem representation

problem = MultiAgentProblem("HRC")

# Agents definition
robot = Agent("robot", problem)
human = Agent("human", problem)
problem.add_agent(robot)
problem.add_agent(human)

# User Types
Location = UserType('Location')
Cube = UserType('Cube')
RobotCube = UserType('RobotCube', Cube)
HumanCube = UserType('HumanCube', Cube)

# Agent_type = UserType('Agent')
object_types = {"human": HumanCube,
               "robot": RobotCube}

# Fluents
object_at = unified_planning.model.Fluent('object_at', BoolType(), obj=Cube, l=Location)
holding = unified_planning.model.Fluent('holding', BoolType(), obj=Cube)
free = unified_planning.model.Fluent('free', BoolType())    # Non prende un Cube come param in input

problem.ma_environment.add_fluent(object_at, default_initial_value=False)
robot.add_private_fluent(holding, default_initial_value=False)                           # default initial value None???, dovrebbe essere false per tutti
robot.add_private_fluent(free, default_initial_value=True)
human.add_private_fluent(holding, default_initial_value=False)                           # default initial value None???, dovrebbe essere false per tutti
human.add_private_fluent(free, default_initial_value=True)


# Objects
locations = [Object(f"{location}", Location) for location in locations_name]
cubes = []

for obj in obj_goals:
  for obj_type in obj_agents:
    if obj_type in obj:
       cubes.append(Object(f"{obj}", object_types[obj_agents[obj_type]]))
       
problem.add_objects(locations)
problem.add_objects(cubes)

# Usefull specific locations
pick_area = next(filter(lambda location: "pick" in location.name, locations),None)
place_area = next(filter(lambda location: "release" in location.name, locations),None)

if not pick_area:
  print("No picking area exist, unable to go on")
  raise Exception("No picking area exist")
if not place_area:
  print("No place area exist, unable to go on")
  raise Exception("No place area exist")

# Robot actions

# Pick action
pick_robot = unified_planning.model.InstantaneousAction('pick', pick_obj=RobotCube) #, l_to=Location)
pick_obj = pick_robot.parameter('pick_obj')

# Precondition
pick_robot.add_precondition(object_at(pick_obj, pick_area))
pick_robot.add_precondition(free)                             #come si fa a dire all(holding, obj) == False
# Effects
pick_robot.add_effect(object_at(pick_obj,pick_area), False)
pick_robot.add_effect(holding(pick_obj), True)
pick_robot.add_effect(free, False)

robot.add_action(pick_robot)

# Place Robot
place_robot = unified_planning.model.InstantaneousAction('place', place_obj=RobotCube) #, l_to=Location)
place_obj = place_robot.parameter('place_obj')

place_robot.add_precondition(holding(place_obj))

# pick_robot.add_precondition(robot_at(l_from))
# pick_robot.add_effect(robot_at(l_from), False)
place_robot.add_effect(object_at(place_obj,place_area), True)
place_robot.add_effect(holding(place_obj), False)
place_robot.add_effect(free, True)

robot.add_action(place_robot)

# Human actions

# Pick action
pick_human = unified_planning.model.InstantaneousAction('pick_h', pick_obj_h=HumanCube) #, l_to=Location)
pick_obj_h = pick_human.parameter('pick_obj_h')

# Precondition
pick_human.add_precondition(object_at(pick_obj_h, pick_area))
pick_human.add_precondition(free)                             #come si fa a dire all(holding, obj) == False
# Effects
pick_human.add_effect(object_at(pick_obj_h,pick_area), False)
pick_human.add_effect(holding(pick_obj_h), True)
pick_human.add_effect(free, False)

human.add_action(pick_human)

# Place Human
place_human = unified_planning.model.InstantaneousAction('place_h', place_obj_h=HumanCube) #, l_to=Location)
place_obj_h = place_human.parameter('place_obj_h')

place_human.add_precondition(holding(place_obj_h))

place_human.add_effect(object_at(place_obj_h,place_area), True)
place_human.add_effect(holding(place_obj_h), False)
place_human.add_effect(free, True)

human.add_action(place_human)


# Initial conditions
for obj in cubes:
  problem.set_initial_value(object_at(obj, pick_area), True)
  # problem.set_initial_value(holding(obj), False)

# Task-Goal
for obj_goal in obj_goals:
  print(obj_goal)
  obj_goal_as_Obj = next(filter(lambda cube: obj_goal in cube.name, cubes),None)
  if obj_goal_as_Obj:
    problem.add_goal(object_at(obj_goal_as_Obj, place_area))

print(problem)

with OneshotPlanner(name='fmap') as planner:
    result = planner.solve(problem, None, "1")
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print("%s Returned Sequential Plans object: %s" % (planner.name, result.plan.all_sequential_plans()))
        # [print(f"{idx} Sequential Plans: {seq_plan}") for idx, seq_plan in enumerate(result.plan.all_sequential_plans())]
        # print("Adjacency list:", result.plan.get_adjacency_list)
        print("result:", result)
    else:
        print("Log Error:", result)


b1
b2
b3
b4
o1
o2
o3
o4
w1
w3
w2
w4
problem name = HRC

types = [Cube, Location, RobotCube - Cube, HumanCube - Cube]

environment fluents = [
  bool object_at[obj=Cube, l=Location]
]

agents = [
  Agent name = robot

private fluents = [
 bool holding[obj=Cube]
 bool free
]

public fluents = [
]

actions = [
 action pick(RobotCube - Cube pick_obj) {
    preconditions = [
      object_at(pick_obj, pick_area)
      free
    ]
    effects = [
      object_at(pick_obj, pick_area) := false
      holding(pick_obj) := true
      free := false
    ]
  }
 action place(RobotCube - Cube place_obj) {
    preconditions = [
      holding(place_obj)
    ]
    effects = [
      object_at(place_obj, release_area) := true
      holding(place_obj) := false
      free := true
    ]
  }
]

private goals = [
]

public goals = [
]


  Agent name = human

private fluents = [
 bool holding[obj=Cube]
 bool free
]

public fluents = [
]

actions = [
 action pick_h(HumanCube - Cube pick_obj_h) {
    preconditions =

FileNotFoundError: [Errno 2] No such file or directory: 'java'

In [5]:
from unified_planning.plot import plot_partial_order_plan
# Redefine the plot package methods imported above to print the plot to a temp file
# if the exception "could not locate runnable browser" is raised. This usually happens
# in the Continuous Integration.

from inspect import getmembers, isfunction
from unified_planning import plot
from functools import partial
import os, uuid, tempfile as tf

# Define the function that will be executed instead
def _function(original_function, *args, **kwargs):
    try:
        original_function(*args, **kwargs)
    except Exception as e:
        if "could not locate runnable browser" in str(e):
            original_function(*args, **kwargs,
                filename=f"{os.path.join(tf.gettempdir(), str(uuid.uuid1()))}.png"
            )
        else:
            raise e

# Iterate over all the functions of the plot package
for function_name, function in getmembers(plot, isfunction):
    # Override the original function with the new one
    globals()[function_name] = partial(_function, function)
plot_partial_order_plan(result.plan, figsize=(20, 20), node_size=8000, font_size=10, top_bottom=True)


NameError: name 'result' is not defined